In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ratings= pd.read_csv('ratings.csv')
movies= pd.read_csv('movies.csv')

# Question 1

Write a MapReduce program that takes the input dataset and outputs the number of unique users who visited each page. Your program should output tuples of the form (movieId, unique_users).

In [ ]:
ratings

Splitting the table into 10 tables

In [ ]:
total_rows = len(ratings)

rows_per_table = total_rows // 10

tables = []
start_row = 0

for i in range(10):
    end_row = start_row + rows_per_table
    table = ratings[start_row:end_row]
    tables.append(table)
    start_row = end_row

for i, table in enumerate(tables):
    print(f"Table {i+1}:")
    print(table)
    print()

Creating a list of (userId,movieId)

In [ ]:
my_list=[]
for table in tables:
    result=[]
    result = list(zip(table['userId'], table['movieId']))
    my_list.append(result)

for i, result in enumerate(my_list):
    print(f"Table {i+1}:")
    print(result)

Giving 1 to each ( userId, movieId)

In [ ]:
dictionaries = []

for result in my_list:
    my_dict = {}
    for key in result:
        my_dict[key] = [1]
    dictionaries.append(my_dict)

# Print the dictionaries
for i, my_dict in enumerate(dictionaries):
    print(f"Table {i+1}:")
    for key, values in my_dict.items():
        print(key, values)
    print()


Here, we can see in each table, how many uniqueIds have watched each movie.

In [ ]:
lastlist = []
for dictionary in dictionaries:
    firstlist = {}
    for key in dictionary.keys():
        new_key = key[1]
        if new_key in firstlist:
            firstlist[new_key].append(1)
        else:
            firstlist[new_key] = [1]
    lastlist.append(firstlist)
    
for i, firstlist in enumerate(lastlist):
    print(f"Table {i+1}:")
    for key, values in firstlist.items():
        print(key, values)
    print()

In [ ]:
lastlist

Now, we collect all the uniqueIds and take them out of the tables.

In [ ]:
def collect_values(dictionaries):
    common_values = {}
    
    for dictionary in dictionaries:
        for key in dictionary:
            if key in common_values:
                common_values[key].append(dictionary[key])
            else:
                common_values[key] = [dictionary[key]]
    
    return common_values


collect= collect_values(lastlist)

In [ ]:
collect

Reducing

In [ ]:
def count_values(A):
    count_dict = {}
    for key, values in A.items():
        count = 0
        for sublist in values:
            count += sublist.count(1)
        count_dict[key] = count
    return count_dict

x = count_values(collect)
print(x)


Creat a tuple of them:

In [ ]:
mytuple = tuple(x.items())
sorted_tuple = sorted(mytuple, key=lambda x: x[1], reverse=True)
sorted_tuple

# Question 2

Write a MapReduce program that takes the input dataset and outputs the top-rated movies. Your program should output tuples of the form (movie_id, average_rating), sorted in descending order by average_rating.

In [ ]:
ratings

In [ ]:
ratings.info()

Splitting the data into 10 new tables

In [ ]:
total_rows = len(ratings)

rows_per_table = total_rows // 10

tables = []
start_row = 0

for i in range(10):
    end_row = start_row + rows_per_table
    table = ratings[start_row:end_row]
    tables.append(table)
    start_row = end_row
#I stored my new tables in "tables".

#showing the tables:
for i, table in enumerate(tables):
    print(f"Table {i+1}:")
    print(table)
    print()


Because of the fact that keys are unique in dictionaries, I gave a number to each of them to make some differences between same keys. result: ( number, movieId, rating)

In [ ]:
my_list=[]
for table in tables:
    result=[]
    numbers = list(range(1, len(table['userId'])))
    result = list(zip(numbers,table['movieId'], table['rating']))
    my_list.append(result)
    
for i, result in enumerate(my_list):
    print(f"Table {i+1}:")
    print(result)

for each rate, I gave 1 as the value.

In [ ]:
dictionaries = []

for result in my_list:
    my_dict = {}
    for key in result:
        my_dict[key] = [1]
    dictionaries.append(my_dict)

# Print the dictionaries
for i, my_dict in enumerate(dictionaries):
    print(f"Table {i+1}:")
    for key, values in my_dict.items():
        print(key, values)
    print()


In [ ]:
dictionaries

Collecting values in each table with the same key

In [ ]:
lastlist = []
for dictionary in dictionaries:
    firstlist = {}
    for key in dictionary.keys():
        new_key = key[1:3]
        if new_key in firstlist:
            firstlist[new_key].append(1)
        else:
            firstlist[new_key] = [1]
    lastlist.append(firstlist)
    
for i, firstlist in enumerate(lastlist):
    print(f"Table {i+1}:")
    for key, values in firstlist.items():
        print(key, values)
    print()

In [ ]:
lastlist

Collecting values with the same keys through all tables

In [ ]:

collected=collect_values(lastlist)
collected

Reducing

In [ ]:
y = count_values(collected)
print(y)

let's calculate the average for each movieId:

In [ ]:
def numofrates(mydict):
    new_dict = {}
    for key, value in mydict.items():
        movieid, rate = key
        num = value
        score= rate*num
        if movieid in new_dict:
            new_dict[movieid].append(score)
        else:
            new_dict[movieid] = [score]

    return new_dict
myoutput = numofrates(y)
print(myoutput)

In [ ]:
averages = {key: sum(values) / len(values) for key, values in myoutput.items()}
averages

The answer in next level will be ready:)

In [ ]:
sorted_dict = {key: value for key, value in sorted(averages.items(), key=lambda item: item[1], reverse=True)}
my_tuple = tuple(sorted_dict.items())
print(my_tuple)


# Question 3

You have to find the movies that are similar to a given movie, based on the MinHash algorithm estimate the Jaccard similarity.

In [ ]:
movies

Creating a dictionary of my dataset

In [ ]:
movie = {}

for index, row in movies.iterrows():
    key = row['title']
    value = row['genres']
    movie[key] = value


print(movie)


In [ ]:
film = str(input("Enter the movie's name:"))

my_vocab = movie[film]
my_vocab

In [ ]:
for key, value in movie.items():
    movie[key]=value.split("|")
movie


In [ ]:
vocab = set()
for g in movie.values():
    vocab.update(g)
vocab = tuple(vocab)

print(vocab)


In [ ]:
df = pd.DataFrame(0, index=vocab, columns=movie.keys())

for movie1, genre1 in movie.items():
    df.loc[genre1, movie1] = 1

# Characteristic matrix
print(df)


In [ ]:
#putting number 0 to 19 instead of the vocab


# Create an empty DataFrame
df2 = pd.DataFrame(0, index=range(len(vocab)), columns=movie.keys())

# Set 1 for movies that have the corresponding genre
for movie1, genre1 in movie.items():
    genre_indices = [vocab.index(genre) for genre in genre1]
    df2.loc[genre_indices, movie1] = 1

df2

In [ ]:

film = df2[film].values

jaccard_similarities = {}
for column in df2.columns:
    if column != film:
        other_film = df2[column].values
        intersection = sum((film[i] == 1) and (other_film[i] == 1) for i in range(len(film)))
        union = sum((film[i] == 1) or (other_film[i] == 1) for i in range(len(film)))
        jaccard_similarities[column] = intersection / union

for film, similarity in jaccard_similarities.items():
    print(film, similarity)


**So, here we can see the similar movies to the gven movie.**

In [ ]:
sorted_dict = {key: value for key, value in sorted(jaccard_similarities.items(), key=lambda item: item[1], reverse=True)}
my_tuple = tuple(sorted_dict.items())
print(my_tuple)


# Question 3, (2nd solution)

At the privious part, I found similar movies using characteristic matrix.
Now, I wanna try doing it using signature matrix.

In [ ]:
def generate_hash_functions(num_perm, num_cols):
    hash_funcs = []
    for _ in range(num_perm):
        hash_func = lambda x, seed=random.randint(1, 2**31-1): hash((x, seed)) % 20
        hash_funcs.append(hash_func)
    return hash_funcs


In [ ]:
def generate_signature(matrix, num_perm):
    num_rows, num_cols = matrix.shape
    hash_funcs = generate_hash_functions(num_perm, num_cols)
    signature = np.full((num_perm, num_cols), np.inf)
    
    for col_idx in range(num_cols):
        for row_idx in range(num_rows):
            if matrix[row_idx, col_idx] == 1:
                for perm_idx in range(num_perm):
                    hash_val = hash_funcs[perm_idx](col_idx)
                    signature[perm_idx, col_idx] = min(signature[perm_idx, col_idx], hash_val)

    return signature


In [ ]:
import random
signature = generate_signature(np.array(df2), 6)
signature
signature.shape

In [ ]:
# Here's my signature
sig = pd.DataFrame(signature, columns=movies['title'][0:9737], index=[i for i in range(6)])
sig

In [ ]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0


In [ ]:
film = df2[film].values

jaccard_similarities = {}
for column in df2.columns:
    if column != film:
        other_film = df2[column].values
        intersection = sum((film[i] == 1) and (other_film[i] == 1) for i in range(len(film)))
        union = sum((film[i] == 1) or (other_film[i] == 1) for i in range(len(film)))
        jaccard_similarities[column] = intersection / union

for film, similarity in jaccard_similarities.items():
    print(film, similarity)


**Finished in 2 different ways:)**

# Question 4

Find the movies that are closest to five arbitrary movie.

**First of all, we have to choose 5 movies. Then, I'm gonna put their genres in a list and continue like as "question 1" to find similar movies.**

In [ ]:
movie

In [ ]:
my_movies = {'Tom and Huck (1995)': ['Adventure', 'Children'],
 'Sudden Death (1995)': ['Action'],
 'GoldenEye (1995)': ['Action', 'Adventure', 'Thriller'],
 'American President, The (1995)': ['Comedy', 'Drama', 'Romance'],
 'Dracula: Dead and Loving It (1995)': ['Comedy', 'Horror']}

vocabs = set()
for g in my_movies.values():
    vocabs.update(g)
vocabs = tuple(vocabs)

vocabs


In [ ]:
movie2 = movie.copy()

I'm going to replacing 5 movies in the dictionary 'my_movies' with '5movies' : vocabs.

In [ ]:
#adding new one
movie2['5movies'] = vocabs
#deleting those 5 movies
for key in my_movies.keys():
        del movie2[key]

movie2

In [ ]:
film = '5movies'

Other steps are like question 3. So, we should calculate characteristic matrix and signature for movie2 and at the end, we wil calculate jaccard_similarity between '5movies' and other movies.

In [ ]:
df3 = pd.DataFrame(0, index=vocab, columns=movie2.keys())

for movie1, genre1 in movie2.items():
    df3.loc[genre1, movie1] = 1

# Characteristic matrix
print(df3)


In [ ]:
#putting number 0 to 19 instead of the vocab
df4 = pd.DataFrame(0, index=range(len(vocab)), columns=movie2.keys())


for movie1, genre1 in movie2.items():
    genre_indices = [vocab.index(genre) for genre in genre1]
    df4.loc[genre_indices, movie1] = 1

df4

In [ ]:
film = df4[film].values

jaccard_similarities2 = {}
for column in df4.columns:
    if column != film:
        other_film = df4[column].values
        intersection = sum((film[i] == 1) and (other_film[i] == 1) for i in range(len(film)))
        union = sum((film[i] == 1) or (other_film[i] == 1) for i in range(len(film)))
        jaccard_similarities[column] = intersection / union

for film, similarity in jaccard_similarities2.items():
    print(film, similarity)


In [ ]:
sorted_dict2 = {key: value for key, value in sorted(jaccard_similarities2.items(), key=lambda item: item[1], reverse=True)}
my_tuple2 = tuple(sorted_dict2.items())
print(my_tuple2)
